In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
train

In [ ]:
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
test

In [ ]:
type(train['Date'][0])

In [ ]:
# Date 컬럼 데이터 타입은 문자형
# 날짜형으로 데이터형식 바꾸기
train['Date'] = pd.to_datetime(train['Date'])
# 또 다른 방법 : train['Date'] = train['Date'].astype('datetime64')
train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day

In [ ]:
# 데이터 군집화
train.groupby('day')['Weekly_Sales'].mean()
# day가 도움이 되는 이유 : 월 정보를 세세하게 나눌 수 있음(월초,월말 세일즈 패턴 학습 가능)
# --> 이 데이터셋에서는 월초에 구매를 많이 하는 것 확인
#   특정 날짜가 중요한 경우가 있음(이벤트 or 프로모션으로 인한 세일즈 증가)

In [ ]:
# test 데이터셋도 동일하게 날짜 처리 해주기!
test['Date'] = test['Date'].astype('datetime64')
test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test

In [ ]:
# Date 코드 제거하기!
# train/test 독립변수(종류/개수)를 동일하게 맞춰주자!
train_2 = train.drop(columns = ['Date','Weekly_Sales'])
train_2

In [ ]:
test_2 = test.drop(columns = 'Date')
test_2

In [ ]:
# Store과 Dept 변수 Binary class로 나눈다?(one-hot encoding)
# -> 모델 점수에도 영향을 줄 수 있으므로 하지 않는다
# --> 그냥 이대로 학습시키자!
# Decision Tree 계열 모델들은 데이터 구분을 알아서 잘 해준다
# 정규화 / 스케일링 ? --> 할 필요 없다 --> Tree모델이 알아서 변수를 확인하고 해준다
# 다만 다른 모델들을 사용할때는 따로 해줘야함!

In [ ]:
%%time
# RandomForest 모델링
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(train_2,train['Weekly_Sales'])
# n_jobs --> cpu 코어를 몇개 쓸것인가?
# 모델학습(독립변수)

In [ ]:
result = rf.predict(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
sub

In [ ]:
sub['Weekly_Sales'] = result
sub

In [ ]:
sub.to_csv('result.csv',index=False)

In [ ]:
# 모델점수 올리기
# 1) 모델옵션/파라미터 세팅 잘하기
# 2) 새로운 유용한 컬럼 추가 --> year/month/day 컬럼 위에 추가해주었음!
# month 컬럼을 추가했더니 평가방식에 의해서 점수가 많이 높아지는 것 확인!
# 회사의 평가방식 및 목적 고려!